# Error Propagation using Random Gaussian Numbers

Example calculation of propagating uncertainties, both when adding and multiplying number, and also in the general case. The propagation can be done both analytically (using the error propagation formula) and also using simulation.

The example is based on FIRST doing the error propagation **analytically**, and then verifying it by running a so-called Monte-Carlo (MC) program, which uses random numbers for propagating errors.

## References:
- Barlow: page 48-61
- Bevington: page 36-48

## Author(s), contact(s), and dates:
- Author: Troels C. Petersen (NBI)
- Email:  petersen@nbi.dk
- Date:   8th of November 2023

***

DO THE FOLLOWING ANALYTICAL EXERCISE FIRST!!!

1. A class of students estimate by eye, that the length of the table in Auditorium A is $L = (3.5\pm 0.4)$m, and that the width is $W = (0.8\pm 0.2)$m.

   Assuming that there is no correlation between these two measurements, calculate ANALYTICALLY what the Perimeter (P), area (A), and diagonal (D) length is including (propagated) uncertainties. Repeat the calculation, given that the correlation between length and width is $\rho(L,W) = 0.5$ - not an unreasonable number, given that they are estimated by the same (uncertain) scale.
   
NOTE: This is a complete standard problem, that you will be asked to solve again and again in the course. For this reason, make sure that you understand how to do it, and become good at doing it reasonably fast.

In [ ]:
mu1   =  3.5
sig1  =  0.4
mu2   =  0.8
sig2  =  0.2
rho12 =  0.0           # Correlation parameter!

Check if the correlation is valid:

In [ ]:
if not (-1.0 <= rho12 <= 1.0): 
    raise ValueError(f"Correlation factor not in interval [-1,1], as it is {rho12:6.2f}")

### Note on analytic solutions with SymPy:

Python includes symbolic algebra in the package *SymPy*, which is both simple and powerful (and in Python). In addition, printing with Latex can also be included (see below), which (in combination) is very nice.

Below is a SymPy and Latex example with the hope that it will wet your appetite.

In [ ]:
from IPython.core.display import Latex

def lprint(*args,**kwargs):
    """Pretty print arguments as LaTeX using IPython display system 
    
    Parameters
    ----------
    args : tuple 
        What to print (in LaTeX math mode)
    kwargs : dict 
        optional keywords to pass to `display` 
    """
    display(Latex('$$'+' '.join(args)+'$$'),**kwargs)

In [ ]:
def myDiff(formula):
    return sqrt((formula.diff(L) * dL)**2 + (formula.diff(W) * dW)**2)

def myDiffWithCorr(formula, name = "", printNow = False):
    dd = sqrt((formula.diff(L) * dL)**2 + (formula.diff(W) * dW)**2 + 2*(formula.diff(L)*formula.diff(W)*(sigCorr**2)))
    if(printNow):
        lprint(latex(Eq(symbols('sigma_'+name), dd)))
    fd = lambdify((L,dL,W,dW,sigCorr),dd)
    return dd, fd
    
def diff_and_print(formula, name = ""):
    # Calculate uncertainty and print original relation/formula and the uncertainty
    dd = myDiff(formula)
    lprint(latex(Eq(symbols(name),formula)))
    lprint(latex(Eq(symbols('sigma_'+name), dd)))
    
def lambdifyFormula(formula, *args, name = ""):
    # Turn expression into numerical functions 
    f = lambdify((L,W),formula)
    d = myDiff(formula)
    fd = lambdify((L,dL,W,dW),d)
    return f, fd

In [ ]:
# Import SymPy: 
from sympy import * 

# Define variables:
L,W,P,A,D = symbols("L, W, P, A, D")
dL,dW,dP,dA,dD = symbols("sigma_L, sigma_W, sigma_P, sigma_A, sigma_D")

# Define relations:
# Perimeter:
P = 2*L + 2*W
# Area:
A = L*W
# Diagonal
D = sqrt(L**2 + W**2)

# Try writing a simple function to not repeat yourself! (See cell above)
diff_and_print(P,"P")
diff_and_print(A,"A")
diff_and_print(D,"D")

dP = myDiff(P)
dA = myDiff(A)
dD = myDiff(D)

# Turn expressions into numerical functions 
fP, fdP = lambdifyFormula(P,"P")
fA, fdA = lambdifyFormula(A,"A")
fD, fdD = lambdifyFormula(D,"D")

# Define values and their errors
vL, vdL = mu1,sig1
vW, vdW = mu2,sig2

# Numerically evaluate expressions and print 
vP = fP(vL,vW)
vdP = fdP(vL,vdL,vW,vdW)
vA = fA(vL,vW)
vdA = fdA(vL,vdL,vW,vdW)
vD = fD(vL,vW)
vdD = fdD(vL,vdL,vW,vdW)

lprint(fr'P = ({vP:.1f} \pm {vdP:.1f})\,\mathrm{{m}}')
lprint(fr'A = ({vA:.1f} \pm {vdA:.1f})\,\mathrm{{m}}')
lprint(fr'D = ({vD:.1f} \pm {vdD:.1f})\,\mathrm{{m}}')

#Adding correlations (and also derivation, printing and lambdifying)
sigCorr = symbols("sigma_LW")
rho = symbols("rho_LW")

dP, fdP = myDiffWithCorr(P, "P", True)
dA, fdA = myDiffWithCorr(A, "A", True)
dD, fdD = myDiffWithCorr(D, "D", True)

sCorr = sqrt(rho*dL*dW)
fSC = lambdify((rho,dL,dW),sCorr)

vSigmaCorr = fSC(rho12,vdL,vdW)

# Numerically evaluate expressions and print 
vdP = fdP(vL,vdL,vW,vdW,vSigmaCorr)
vdA = fdA(vL,vdL,vW,vdW,vSigmaCorr)
vdD = fdD(vL,vdL,vW,vdW,vSigmaCorr)

lprint(fr'P = ({vP:.1f} \pm {vdP:.1f})\,\mathrm{{m}}')
lprint(fr'A = ({vA:.1f} \pm {vdA:.1f})\,\mathrm{{m}}')
lprint(fr'D = ({vD:.1f} \pm {vdD:.1f})\,\mathrm{{m}}')

In [ ]:
import numpy as np                                     # Matlab like syntax for linear algebra and functions
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
import seaborn as sns                                  # Make the plots nicer to look at
from iminuit import Minuit                             # The actual fitting tool, better than scipy's
import sys                                             # Modules to see files and folders in directories

In [ ]:
sys.path.append('../../External_Functions')
from ExternalFunctions import Chi2Regression
from ExternalFunctions import nice_string_output, add_text_to_ax # useful functions to print fit results on figure

## Error propagation - Simulation

Now we want to try to see, if we can solve the above error propagation problem using simulation. The method is relatively straight forward: You simply take "realistic" values of the input parameters x (here Length (x1) and Width (x2)), calculate the resulting value y (here Perimeter, Area, and Diagonal), and do this many times. The resulting distribution of y should be centered around the value y(x1,x2), and the standard deviation should reflect the uncertainty in y from the uncertainties in the input variables.

This is a much more clumsy way of calculating the uncertainty, but comes with the advantage, that if the resulting uncertainty is not Gaussian, then one can actually see this (i.e. it avoids the assumptions used in the usual error propagation formula).

In [ ]:
# First we set the parameters of the program:
N_exp = 10000           # Number of "experiments" (i.e. drawing from random distributions)
save_plots = False
r = np.random
r.seed(42)

#### Producing random numbers with a correlation:

Below we have used the build in Numpy method for producing two random Gaussian numbers with a correation between them.

You can also do this "yourself", see Barlow page 42-44 for method. Essentially, the method is to generate uncorrelated Gaussian numbers, and then "rotating" these, where the amount of rotation controls the correlation wanted.

In [ ]:
# Produce random numbers with (a possible) correlation:
cov = np.array([[sig1**2, rho12*sig1*sig2],
                [rho12*sig1*sig2, sig2**2]])
x12_all = np.random.multivariate_normal([mu1, mu2], cov, size=N_exp)

# Now we use the input variables (x1 and x2) to calculate y:
y_all_P = 2*x12_all[:,0] + 2*x12_all[:,1]               # Perimeter (P)
y_all_A = x12_all[:,0] * x12_all[:,1]                   # Area (A)
y_all_D = np.sqrt(x12_all[:,0]**2 + x12_all[:,1]**2)    # Diagonal (D)

***
Plot both input distribution and resulting 2D-histogram on screen:

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))
counts, xedges, yedges, im = ax.hist2d(x12_all[:,0], x12_all[:,1], bins=[120, 80], range=[[0.0, 6.0], [-1.0, 3.0]], cmin=1)
ax.plot([0.0, 6.0], [0.0, 0.0], "--k")    # NOTE: Dashed black line from [x1, x2], [y1, y2] with dashed line
fig.colorbar(im)

ax.set(title='Histogram of lengths (x) and widths (y)',
       xlabel='x', 
       ylabel='y'
      )

d = {'Entries': len(x12_all),
     'Mean x': x12_all[:,0].mean(),
     'Mean y': x12_all[:,1].mean(),
     'Std x': x12_all[:,0].std(ddof=1),
     'Std y': x12_all[:,1].std(ddof=1),
    }

text = nice_string_output(d, extra_spacing=2, decimals=3)
add_text_to_ax(0.02, 0.97, text, ax, fontsize=15);

fig.tight_layout()
fig

if save_plots :
    fig.savefig("Dist_2Dgauss.pdf")

## Looking at the resulting distribution:

Now we look at the `y_all` distribution, which should be Gaussian if the error propagation formula holds.

However, we are in principle not even sure, if this distribution is Gaussian, so in order to check this, we draw a Gaussian on top using the values obtained analytically above.

In [ ]:
def gaussian(x, binwidth, N, mu, sigma):
    return N * binwidth / (sigma*np.sqrt(2*np.pi)) * np.exp(-0.5* (x-mu)**2/sigma**2)

In [ ]:
# Perimeter:
# ----------
# Always make sure, that you control the binning and range!
nbins = 100
xmin, xmax = 5.0, 15.0
binwidth = (xmax-xmin)/nbins
print(f"  Mean = {y_all_P.mean():5.3f},    Std = {y_all_P.std(ddof=1):5.3f}")

figP, axP = plt.subplots(figsize=(16, 6));
countsP, bin_edgesP, _ = axP.hist(y_all_P, nbins, range=(xmin, xmax), histtype='step', linewidth=2)
bin_centersP = (bin_edgesP[1:] + bin_edgesP[:-1])/2
s_countsP = np.sqrt(countsP)

# Draw Gaussian from analytical calculation on top:
xaxis = np.linspace(xmin, xmax, 1000)
yaxis = gaussian(xaxis, binwidth, N_exp, vP, vdP)
axP.plot(xaxis, yaxis, linewidth=2)

figP.tight_layout()
figP

if save_plots:
    fig2.savefig("Dist_ErrorProp_Perimeter.pdf")

In [ ]:
# Area:
# -----
# Always make sure, that you control the binning and range!
nbins = 100
xmin, xmax = 0.0, 10.0
binwidth = (xmax-xmin)/nbins
print(f"  Mean = {y_all_A.mean():5.3f},    Std = {y_all_A.std(ddof=1):5.3f}")

figA, axA = plt.subplots(figsize=(16, 6));
countsA, bin_edgesA, _ = axA.hist(y_all_A, nbins, range=(xmin, xmax), histtype='step', linewidth=2)
bin_centersA = (bin_edgesA[1:] + bin_edgesA[:-1])/2
s_countsA = np.sqrt(countsA)

# Draw Gaussian from analytical calculation on top:
xaxis = np.linspace(xmin, xmax, 1000)
yaxis = gaussian(xaxis, binwidth, N_exp, vA, vdA)
axA.plot(xaxis, yaxis, linewidth=2)

figP.tight_layout()
figP

if save_plots:
    fig2.savefig("Dist_ErrorProp_Area.pdf")

In [ ]:
# Diagonal:
# ---------
# Always make sure, that you control the binning and range!
nbins = 100
xmin, xmax = 0.0, 5.0
binwidth = (xmax-xmin)/nbins
print(f"  Mean = {y_all_D.mean():5.3f},    Std = {y_all_D.std(ddof=1):5.3f}")

figD, axD = plt.subplots(figsize=(16, 6));
countsD, bin_edgesD, _ = axD.hist(y_all_D, nbins, range=(xmin, xmax), histtype='step', linewidth=2)
bin_centersD = (bin_edgesD[1:] + bin_edgesD[:-1])/2
s_countsD = np.sqrt(countsD)

# Draw Gaussian from analytical calculation on top:
xaxis = np.linspace(xmin, xmax, 1000)
yaxis = gaussian(xaxis, binwidth, N_exp, vD, vdD)
axD.plot(xaxis, yaxis, linewidth=2)

figP.tight_layout()
figP

if save_plots:
    fig2.savefig("Dist_ErrorProp_Diagonal.pdf")

---

# Questions:

0. First solve the problem of obtaining the Perimeter, Area & Diagonal with uncertainty ANALYTICALLY.

**Example answer 0**: 

See cell 4. No need to differentiate by hand, SymPy can do it easily for you.

---

1. Now look at the program, and assure yourself that you understand what is going on. Put in the correct expression for y in terms of x1=L and x2=W in order to calculate the perimeter, area, and diagonal length, and run the program. Does the output correspond well with the results you expected from your analytical calculations to begin with?

**Example answer 1**:
Yes, both the mean value and the uncertainty are in a very good agreement to the results from analytical calculations. In the area calculation, there is a slight discrepancy, simply because multiplying two Gaussians does not give (exactly) a Gaussian... only when adding! 
Also, one little thing to note is, that the width can actually come out negatively in rare cases (i.e. beyond 4 sigma low). When producing a simulation of cases, one should of course be aware of such things!

---

2. Imagine that you wanted to know the central value and uncertainty of y1 and y2, given the
   same above PDFs for `x1`=$L$ and `x2`=$W$:
   
     `y1 = log(square(x1*tan(x2))+sqrt((x1-x2)/(cos(x2)+1.0+x1)))`
     
     `y2 = 1.1+sin(20*x1)`

   Get the central value of y, and see if you can quickly differentiate this with
   respect to `x1` and `x2`, and thus predict what uncertainty to expect for y using
   the error propagation formula. It is (for once) OK to give up on the first expression :-)
   Next, try to estimate the central value and uncertainty using random numbers
   like above - do you trust this result more? And are the distributions Gaussian?

**Example answer 2**:
The first expression "y1" looks horrible, and it is! This is an obvious case where simulation is probably a better way forward, and at least a good cross check.
The second expression "y2" is simple, but made to make the error propagation formula break down! The function varies wildly, and as a result, the error propagation formula assumption is broken (badly). The resulting error distribution is also very far from Gaussian, as can be seen below.

For code solution example, see cell 19 below.


### Advanced questions:

3. Try to generate `x1` and `x2` with non-linear correlation, which yields zero linear correlation,
   and see that despite not having any linear correlation, the result on perimeter, area, and diagonal
   length is still affected.

# Learning points:

Through this exercise, you should understand, that uncertainties (errors in slang) propagation can be in **two ways**:
1. **Analytically**, propergating the uncertainties by differentiating the formula/relation in question.
2. **Numerically** (i.e. simulation), by using random numbers reflecting the uncertainties on the input parameters, and calculating the final resulting number many times from these, noting the variation.

You should be capable of **using both methods** effortlessly and with confidence.

The analytical method is simple and transparent, but not always robust, as it requires that the error propagation formula holds. The numerical method is simple and robust, but not transparent. Using both methods is a great way of cross checking.

Finally, you should understand, that error propagation plays an essential role in science, and that it is also used in planning of experiments (to minimise the error on the final quantity of interest).